In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Authenticate and mount GCP bucket
from google.colab import auth
auth.authenticate_user()
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse
!mkdir bucket
!gcsfuse retail-video-bucket /content/bucket

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  24185      0 --:--:-- --:--:-- --:--:-- 24185
OK
53 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 53 not upgraded.
Need to get 4,274 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 133872 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.28.1_amd64.deb ...
Unpacking gcsfuse (0.28.1) ...
Setting up gcsfuse (0.28.1) ...
Using mount point: /content/bucket
Opening GCS connection...
Opening bucket...
Mounting file system...
File system has been successfully mounted.


In [0]:
import pandas as pd

with open('/content/bucket/output2.txt', 'r') as filehandle: 
  line_list = filehandle.readlines()
  
import pandas as pd

line_list = [line.replace('[','') for line in line_list]
line_list = [line.replace(']','') for line in line_list]
line_list = [line.split() for line in line_list]


df = pd.DataFrame(line_list,columns=['frame','person_id','bbox_left','bbox_top','bbox_width','bbox_height'])
df.to_csv('/content/bucket/output2.csv')

output_df = pd.read_csv('bucket/output2.csv')
output_df['height_corrected'] = output_df.bbox_height - output_df.bbox_top # bbox_height is mislabled, it's actually the bottom and not a height at all
output_df.groupby('person_id')['height_corrected'].mean()

person_id
1     98.413768
2    103.981755
3    113.027874
4    149.053276
Name: height_corrected, dtype: float64

In [0]:
# this cell from https://stackoverflow.com/questions/28723670/intersection-over-union-between-two-detections

import numpy as np

def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [0]:
test_array = np.array([output_df.iloc[0].bbox_left,output_df.iloc[0].bbox_left + output_df.iloc[0].bbox_width, output_df.iloc[0].bbox_top, output_df.iloc[0].bbox_top - output_df.iloc[0].bbox_height])
test_dict = {'x1': output_df.iloc[0].bbox_left, 'x2': output_df.iloc[0].bbox_width, 'y1': output_df.iloc[0].bbox_top, 'y2': output_df.iloc[0].bbox_height}
IoU(test_array,test_array)
get_iou(test_dict,test_dict)

In [0]:
output_df.iloc[0].bbox_left,output_df.iloc[0].bbox_width, output_df.iloc[0].bbox_top, output_df.iloc[0].bbox_height

In [0]:
for idx,row in output_df[output_df.frame == 3].iterrows():
  print(row.bbox_top)

In [0]:
import cv2

cap = cv2.VideoCapture("/content/gdrive/My Drive/Retail Analytics Data/Video_Set_2/ID_23/Camera_5/Seq_1/Seq_1.avi")
#cap = cv2.VideoCapture("/content/gdrive/My Drive/Retail Analytics Data/Video_Set_1/ID_14/Camera_4/Seq_3/Seq_3.avi")

frame_id = 1

fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
clip_width = 192
clip_height = 384
out = cv2.VideoWriter("/content/gdrive/My Drive/activities2.avi",fourcc, 15.0,(frame_width,frame_height))
#out = cv2.VideoWriter("/content/gdrive/My Drive/activities3.avi",fourcc, 15.0,(frame_width,frame_height))


while True:
  print(frame_id)		

  ret,frame = cap.read()
  if ret is False:
    frame_id+=1
    break	

  for idx,row in output_df[output_df.frame == frame_id].iterrows():
    prev_row3 = output_df[(output_df.frame == row.frame - 3) & (output_df.person_id == row.person_id)]
    prev_row6 = output_df[(output_df.frame == row.frame - 6) & (output_df.person_id == row.person_id)]
    bbox_dict = {'x1': row.bbox_left, 'x2': row.bbox_width, 'y1': row.bbox_top, 'y2': row.bbox_height}
    if prev_row3.size > 0 and prev_row6.size > 0:
      bbox_dict3 = {'x1': prev_row3.bbox_left.values[0], 'x2': prev_row3.bbox_width.values[0], 'y1': prev_row3.bbox_top.values[0], 'y2': prev_row3.bbox_height.values[0]}
      bbox_dict6 = {'x1': prev_row6.bbox_left.values[0], 'x2': prev_row6.bbox_width.values[0], 'y1': prev_row6.bbox_top.values[0], 'y2': prev_row6.bbox_height.values[0]}
      iou03 = get_iou(bbox_dict,bbox_dict3)
      iou06 = get_iou(bbox_dict,bbox_dict6)
      iou36 = get_iou(bbox_dict3,bbox_dict6)
      avg_iou = (iou03 + iou06 + iou36) / 3
    elif prev_row3.size > 0:
      bbox_dict3 = {'x1': prev_row3.bbox_left.values[0], 'x2': prev_row3.bbox_width.values[0], 'y1': prev_row3.bbox_top.values[0], 'y2': prev_row3.bbox_height.values[0]}
      avg_iou = get_iou(bbox_dict,bbox_dict3)
    elif prev_row6.size > 0:
      bbox_dict6 = {'x1': prev_row6.bbox_left.values[0], 'x2': prev_row6.bbox_width.values[0], 'y1': prev_row6.bbox_top.values[0], 'y2': prev_row6.bbox_height.values[0]}
      avg_iou = get_iou(bbox_dict,bbox_dict6)
    else:
      avg_iou = 0 # no previous frames, assume they're moving

    # CORRECTION NEEDED: round instead of int()
    if row.height_corrected < 125: # sitting (RED) (70 for 1st video, 125 for 2nd, 50 for 3rd)
      cv2.rectangle(frame,(int(row.bbox_left),int(row.bbox_top)),(int(row.bbox_width),int(row.bbox_height)),(0,0,255),2)
    elif avg_iou > 0.9: # standing (BLUE)
      cv2.rectangle(frame,(int(row.bbox_left),int(row.bbox_top)),(int(row.bbox_width),int(row.bbox_height)),(255,0,0),2)
    else: # moving (GREEN)
      cv2.rectangle(frame,(int(row.bbox_left),int(row.bbox_top)),(int(row.bbox_width),int(row.bbox_height)),(0,255,0),2)
    cv2.putText(frame, str(row.person_id),(int(row.bbox_left), int(row.bbox_top)),0, 5e-3 * 200, (0,0,0),2)
    
  out.write(frame)
  frame_id+=1

out.release()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
row

Unnamed: 0          1660.000000
frame                300.000000
person_id              3.000000
bbox_left              4.101068
bbox_top              71.838522
bbox_width            37.879287
bbox_height          178.059545
height_corrected     106.221023
Name: 1660, dtype: float64